In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
# !./ngrok authtoken YOUR_NGROK_TOKEN_HERE

In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [1]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [2]:
import builtins
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return builtins.round(builtins.sum(precisions)/len(precisions),3)

In [3]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://2362-35-196-208-168.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      print(pred_wids)
      # print(_)
      ap = average_precision(true_wids, pred_wids)
      print(ap)

  except:
    print('fck')
    pass
  
  qs_res.append((q, duration, ap))

(13213953, 8005736, 57197811, 5377129, 30796675, 48407627, 8778894, 46332325, 21965279, 34192383, 21727049, 18730910, 16305806, 18805500, 696712, 3274540, 645025, 26147929, 28344538, 44783363, 1046562, 3761499, 13584316, 24193838, 36349077, 25061839, 21356332, 58429169, 46448252, 29772882, 14985517, 5087621, 16274121, 35693777, 57836462, 32910103, 750633, 3257046, 47337433, 50020765, 42369403, 10849171, 6823999, 31217917, 1270266, 3596573, 60116836, 24193668, 9603954, 7680006, 17369731, 2009536, 13370873, 8573840, 31354866, 30722935, 3526151, 4195298, 47063809, 49703421, 17940394, 43626386, 4225907, 22442342, 41170321, 30714017, 34292335, 3673376, 51218917, 1255122, 1967274, 12468136, 27319859, 639888, 23862, 18384111, 1810666, 2032271, 2901907, 9687715, 6081823, 32018084, 343952, 13207547, 17474156, 39913402, 56115363, 2852798, 47640599, 794677, 3559817, 2843493, 55654231, 890353, 13371283, 1984246, 24737156, 56391602, 59662859, 46315592)
0.196
(5123787, 5129572, 5129831, 5142364, 516

In [4]:
qs_res

[('python', 6.645523548126221, 0.196),
 ('data science', 4.716379880905151, 0.0),
 ('migraine', 0.7973802089691162, 0.0),
 ('chocolate', 0.8872582912445068, 0.294),
 ('how to make pasta', 3.985795021057129, 0.0),
 ('Does pasta have preservatives?', 0.8884155750274658, 0.0),
 ('how google works', 5.00976824760437, 0.062),
 ('what is information retrieval', 2.401106595993042, 0.138),
 ('NBA', 2.5231659412384033, 0.071),
 ('yoga', 1.0862696170806885, 0.236),
 ('how to not kill plants', 1.7471637725830078, 0.0),
 ('masks', 2.2803895473480225, 0.238),
 ('black friday', 2.9176242351531982, 0.0),
 ('why do men have nipples', 3.5920825004577637, 0.0),
 ('rubber duck', 1.188371181488037, 0.0),
 ('michelin', 1.0865161418914795, 0.026),
 ('what to watch', 2.958415985107422, 0.0),
 ('best marvel movie', 4.639067888259888, 0.0),
 ('how tall is the eiffel tower', 2.88714861869812, 0.043),
 ('where does vanilla flavoring come from', 2.12264347076416, 0.267),
 ('best ice cream flavour', 5.859097003936